In [36]:

import pandas as pd
import torch
import torch.nn.functional as F
import torch.nn as nn
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import time
from sklearn.preprocessing import PolynomialFeatures


torch.cuda.is_available()

True

In [37]:
Data =pd.read_csv("housing.csv")
X = Data.drop(['median_house_value'] , axis=1)
Y = Data['median_house_value']
Y = Y.values.reshape(-1,1)
X


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,NEAR BAY
...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,INLAND


In [38]:
Place=['INLAND','<1H OCEAN','NEAR OCEAN','NEAR BAY','ISLAND']

encoder = OrdinalEncoder(
    categories=[Place],
    handle_unknown='use_encoded_value',
    unknown_value=-1
)
cat_cols = ['ocean_proximity']
new_cols = ['Place']
encoder.fit(X[cat_cols])

X[new_cols] = encoder.transform(X[cat_cols])

X = X.select_dtypes(include=['number'])



X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.20,random_state=131)


median_value_train = X_train.median(numeric_only = True)
X_train = X_train.fillna(median_value_train)

median_value_test =  X_test.median(numeric_only = True)
X_test = X_test.fillna(median_value_test)


poly = PolynomialFeatures(degree = 2,include_bias = False)
X_poly = poly.fit_transform(X_train)
poly_features = pd.DataFrame(X_poly, columns = poly.get_feature_names_out(X_train.columns))
X_train = poly_features
poly_test = PolynomialFeatures(degree = 2,include_bias = False)
X_poly_2 = poly_test.fit_transform(X_test)
X_test = X_poly_2

X_test

#X_val , X_final ,Y_val,Y_final = train_test_split(X_test, Y_test, test_size=0.5, random_state=42)

array([[-118.44      ,   34.15      ,   37.        , ...,   10.76692969,
           3.2813    ,    1.        ],
       [-122.47      ,   37.78      ,   52.        , ...,   17.51924736,
          12.5568    ,    9.        ],
       [-117.24      ,   32.98      ,    4.        , ...,   58.29017104,
          15.2696    ,    4.        ],
       ...,
       [-117.68      ,   33.47      ,    7.        , ...,   37.527876  ,
           6.126     ,    1.        ],
       [-121.03      ,   37.68      ,   27.        , ...,   14.33152449,
           0.        ,    0.        ],
       [-118.11      ,   33.91      ,   29.        , ...,   24.61052881,
           4.9609    ,    1.        ]], shape=(4128, 54))

In [39]:
X_te,X_val,Y_te,Y_val = train_test_split(X_test,Y_test,test_size=0.50,random_state=131)

In [40]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_te = scaler.transform(X_te)
X_val = scaler.transform(X_val)
y_scaler = StandardScaler()
y_scaler.fit(Y_train)
Y_train = y_scaler.transform(Y_train)
Y_te = y_scaler.transform(Y_te)
Y_val= y_scaler.transform(Y_val)
X_test

C:\Users\HP\PycharmProjects\PythonProject\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\HP\PycharmProjects\PythonProject\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[-118.44      ,   34.15      ,   37.        , ...,   10.76692969,
           3.2813    ,    1.        ],
       [-122.47      ,   37.78      ,   52.        , ...,   17.51924736,
          12.5568    ,    9.        ],
       [-117.24      ,   32.98      ,    4.        , ...,   58.29017104,
          15.2696    ,    4.        ],
       ...,
       [-117.68      ,   33.47      ,    7.        , ...,   37.527876  ,
           6.126     ,    1.        ],
       [-121.03      ,   37.68      ,   27.        , ...,   14.33152449,
           0.        ,    0.        ],
       [-118.11      ,   33.91      ,   29.        , ...,   24.61052881,
           4.9609    ,    1.        ]], shape=(4128, 54))

In [41]:
X_train_tensor = torch.tensor(X_train,dtype = torch.float32)
X_test_tensor = torch.tensor(X_te,dtype = torch.float32)
X_val_tensor = torch.tensor(X_val,dtype = torch.float32)
Y_train_tensor = torch.tensor(Y_train,dtype = torch.float32).view(-1,1)
Y_test_tensor = torch.tensor(Y_te,dtype = torch.float32).view(-1,1)
Y_val_tensor = torch.tensor(Y_val,dtype = torch.float32).view(-1,1)
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)
Val_dataset = TensorDataset(X_val_tensor, Y_val_tensor)
train_loader = DataLoader(train_dataset,batch_size=64,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=64)
val_loader = DataLoader(Val_dataset,batch_size=64)

In [42]:
NUM_EPOCHS =100
RANDOM_SEED = 42
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [43]:
def weight_inti(m):
    if isinstance(m,nn.Linear):
        nn.init.kaiming_normal_(m.weight,mode = 'fan_in',nonlinearity = 'relu')
        if m.bias is not None:
            nn.init.constant_(m.bias,0)




In [44]:
class Model(torch.nn.Module):
    def __init__(self,num_features):
        super().__init__()
        self.my_net = torch.nn.Sequential(
            torch.nn.Linear(in_features=num_features, out_features=70),
            torch.nn.BatchNorm1d(num_features=70),
            torch.nn.ELU(),

            torch.nn.Linear(in_features=70, out_features=110),
            torch.nn.BatchNorm1d(num_features=110),
            torch.nn.ELU(),

            torch.nn.Linear(in_features=110, out_features=70),
            torch.nn.BatchNorm1d(num_features=70),
            torch.nn.ELU(),

            torch.nn.Linear(in_features=70, out_features=40),
            torch.nn.BatchNorm1d(num_features=40),
            torch.nn.ELU(),

            torch.nn.Linear(in_features=40, out_features=1),


        )
    def forward(self, x):
        out = self.my_net(x)
        return out


torch.manual_seed(RANDOM_SEED)
model = Model(num_features=54)
model.apply(weight_inti)
model = model.to(DEVICE)



optimizer = torch.optim.AdamW(model.parameters(), lr=0.001,weight_decay=0.01)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,mode = 'min', factor =  0.1 ,patience= 5)


def compute_loss(net,dataloader):
    curr_loss = 0
    with torch.no_grad():
        for cnt , (fet, output) in enumerate(dataloader):
            fet = fet.view((-1,54)).to(DEVICE)
            output = output.view((-1,1)).to(DEVICE)
            out = net(fet)
            loss_c = F.mse_loss(out,output)
            curr_loss += loss_c
        return float(curr_loss)/len(dataloader)

start_time = time.time()
minibatch_cost = []
epoch_cost = []

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx , (features , targets) in enumerate(train_loader):
        features = features.view(-1,54).to(DEVICE)
        targets = targets.to(DEVICE)
        pred = model.forward(features)
        cost = F.mse_loss(pred,targets.float().view(-1,1))
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        minibatch_cost.append(cost.item())
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f'
                   %(epoch+1, NUM_EPOCHS, batch_idx,
                     len(train_loader), cost.item()))
    model.eval()
    Val_loss = 0.0

    with torch.no_grad():
        for inputs_val, targets_val in val_loader:
            inputs_val = inputs_val.view(-1,54).to(DEVICE)
            targets_val = targets_val.to(DEVICE)
            output = model.forward(inputs_val)
            v_loss = F.mse_loss(output,targets_val.float().view(-1,1))
            Val_loss += v_loss.item()

    avg_val_loss = Val_loss / len(val_loader)
    scheduler.step(avg_val_loss)

    cost = compute_loss(model, train_loader)
    epoch_cost.append(cost)
    print('Epoch: %03d/%03d Train Cost: %.4f' % (
            epoch+1, NUM_EPOCHS, cost))
    print('Validation Cost: %.4f' % avg_val_loss)
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))


print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))










Epoch: 001/100 | Batch 000/258 | Cost: 2.6558
Epoch: 001/100 | Batch 050/258 | Cost: 0.2947
Epoch: 001/100 | Batch 100/258 | Cost: 0.1938
Epoch: 001/100 | Batch 150/258 | Cost: 0.2842
Epoch: 001/100 | Batch 200/258 | Cost: 0.3838
Epoch: 001/100 | Batch 250/258 | Cost: 0.3632
Epoch: 001/100 Train Cost: 0.2819
Validation Cost: 0.2665
Time elapsed: 0.04 min
Epoch: 002/100 | Batch 000/258 | Cost: 0.2799
Epoch: 002/100 | Batch 050/258 | Cost: 0.2545
Epoch: 002/100 | Batch 100/258 | Cost: 0.3333
Epoch: 002/100 | Batch 150/258 | Cost: 0.2665
Epoch: 002/100 | Batch 200/258 | Cost: 0.2933
Epoch: 002/100 | Batch 250/258 | Cost: 0.2501
Epoch: 002/100 Train Cost: 0.2583
Validation Cost: 0.2453
Time elapsed: 0.08 min
Epoch: 003/100 | Batch 000/258 | Cost: 0.2093
Epoch: 003/100 | Batch 050/258 | Cost: 0.2217
Epoch: 003/100 | Batch 100/258 | Cost: 0.3480
Epoch: 003/100 | Batch 150/258 | Cost: 0.2179
Epoch: 003/100 | Batch 200/258 | Cost: 0.3908
Epoch: 003/100 | Batch 250/258 | Cost: 0.1688
Epoch: 003

In [46]:
def Test(model,dataloader):
    model.eval()
    avg_te_loss = 0.0
    with torch.no_grad():
        for inputs_te, targets_te in dataloader:

            inputs_te = inputs_te.view(-1,54).to(DEVICE)
            targets_te = targets_te.to(DEVICE)
            output = model.forward(inputs_te)
            te_loss = F.mse_loss(output,targets_te.float().view(-1,1))
            avg_te_loss += te_loss.item()

    avg_te_loss = avg_te_loss / len(test_loader)
    return avg_te_loss

Test(model,test_loader)



0.19753837608026736